<a href="https://colab.research.google.com/github/MandbeZ/TFM_sequia/blob/main/notebooks/4_0_Modelo_Univariante_ARIMA_RF__SPI_SPEI(Por_cluster).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de SKtime

In [ ]:
%pip install sktime
%pip install sktime[all_extras]
%pip install esig
%pip install utils
%pip install matplotlib==3.1.1

In [ ]:
from warnings import simplefilter
simplefilter(action="ignore", category=RuntimeWarning)
simplefilter(action="ignore", category=FutureWarning)
# simplefilter(action="ignore", category=ModelFitWarning)

Importar librerías básicas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sktime.utils.plotting import plot_series
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.compose import make_reduction
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error,\
                                                   mean_absolute_error, mean_squared_error

# Naive Forecast / Pronóstico Ingenuo
from sktime.forecasting.naive import NaiveForecaster

# AutoARIMA
from sktime.forecasting.arima import AutoARIMA

# AutoETS
from sktime.forecasting.ets import AutoETS

# Algoritmos de Regresión de sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
t0 = pd.Timestamp.now()

In [ ]:
#Definición de funciones

def evaluar_modelo(test,pred):
  print('MAPE:',mean_absolute_percentage_error(test, pred, symmetric=False)) 
  print('SMAPE:',mean_absolute_percentage_error(test, pred)) #symmetric default=True
  print('MAE:',mean_absolute_error(test, pred) )
  print('RMSE:',mean_squared_error(test, pred, square_root=True) ) # square_root=True RMSE , false MSE)
  print('MSE:',mean_squared_error(test, pred, square_root=False) ) # square_root=True RMSE , false MSE)

def graficar_modelo(train,test,pred,titulo='Modelo',inicio_serie=200,etiqy='Valor'):
  plot_series(train[inicio_serie:], test, pred, labels=["y_train", "y_test", "y_pred"])
  plt.axhline(y=0.5, color='r', linestyle='dashed')
  plt.title(titulo, c='darkblue', fontweight='bold', fontsize=15)
  plt.ylabel(etiqy.upper(), fontweight='bold', fontsize=12)
  plt.show()
  #plt.savefig('../datos/4.1.Resultados_Graficas/'+titulo.replace(' ','')+'_'+etiqy.split(' ')[0]+'.png', dpi=300)


def graficar_pred_est(data, modelo, clust=[0,1,2,3,4], escala=[1,3,6,12]):
  predic = data

  if 'spi' in predic.columns[0] : s = 'spi'
  else : s = 'spei'

  for e in escala:
    indice = [ind for ind in predic.columns if s+str(e) in ind]
    print('Escala: '+str(e))
    if not indice: 
      print(indice)
      continue
    else:
      datos = cargar_datos('indices_'+s+str(e)+'.csv')
      p_datos = procesa_datos(datos)
      datos_normalizados=normalizar_datos(p_datos)

      for c in clust:
        
        cad = s+str(e)+'_c'+str(c)
        
        if (cad not in predic.columns):
          continue
        else:
          estaciones = cluster[cluster['cluster'] == c].reset_index(drop=True)
          

          for est in range(estaciones.shape[0]):
            estacion = [col for col in datos_normalizados.columns if str(estaciones['id'][est]) in col]
            serie = datos_normalizados.loc[:,estacion]
            train, test = dividir_datos(serie, tamanio = tam_ypred)
            pred = data.iloc[:,0]
            pred.index = test.index
            nom_est = 'Estacion '+str(estaciones.iloc[est,0])+' - '+str(estaciones.iloc[est,1]).capitalize() 
            nom_clu = 'Cluster ' + str(c) + ' - '

            graficar_modelo(train, test, pred,titulo=modelo+' - '+nom_clu+nom_est, inicio_serie=0, etiqy = s+str(e)+' - Normalizado')

# 1. Cargar los datos de SPI

In [ ]:
def cargar_datos(archivo):
    data = pd.read_csv('https://raw.githubusercontent.com/MandbeZ/TFM_sequia/main/datos/spi_spei/' + archivo,  sep = ',', parse_dates=True)
    return data

# 2. Preprocesamiento de los datos

In [ ]:
'''Llevar la  fecha a índice, configurar como periodo'''
def procesa_datos(data):
    data['fecha'] = pd.to_datetime(data['fecha'])
    data = data.dropna()
    data = data.set_index('fecha')
    data.index = data.index.to_period('M')
    return data

## Normalizar los datos

In [ ]:
'''para mejores resultados se pueden normalizar los datos: [0,1]'''
def normalizar_datos(data):
    return (data - data.min()) / ( data.max() - data.min())

## Generar cluster

In [ ]:
'''Devuelve un Dataframe con todas las estaciones de un determinado cluster'''
def gen_cluster(lista_clust, clust, normalizados):
    estaciones = lista_clust[lista_clust['cluster'] == clust]
    nom_cols = [col for est in estaciones['id'] for col in normalizados.columns if str(est) in col]
    datos = normalizados[nom_cols]
    datos = datos.melt(value_name='valor').reset_index(drop=True)
    return datos.iloc[:, 1]

# 3. Dividir Datos en Entrenamiento y prueba

In [ ]:
'''Dividir el Dataset en Entrenamiento y prueba'''
def dividir_datos(serie, tamanio):
    y_train, y_test = temporal_train_test_split(serie, test_size=tamanio)
    return y_train, y_test

## Definir  el horizonte de predicción

In [ ]:
'''Definir horizonte de predicción'''
def horizon_prediccion(y_test):
    return ForecastingHorizon(y_test.index, is_relative=False)  

# 4. Configuración del algoritmo

In [ ]:
def config_algoritmo(algoritmo, estacionalidad = 12, ventana = 12, estrategia = 'mean'):
    if algoritmo=='naive': return NaiveForecaster(strategy=estrategia, sp=estacionalidad, window_length=ventana) #strategy="drift" o "last"  sp estacionalidad
    elif algoritmo=='arima': return AutoARIMA(sp=estacionalidad, suppress_warnings=True) #Cambiar sp para reducir el error
    elif algoritmo=='ets': return AutoETS(auto=True,sp=estacionalidad, n_jobs=-1) #sp=12

def config_algoritmoReg(algoritmo,ventana = 12, estrategia = 'recursive',estimadores=30, vecinos=11):
    if algoritmo=='LinearRegression':
        reg=LinearRegression()    
    elif algoritmo=='DecisionTreeRegressor': 
        reg=DecisionTreeRegressor()
    elif algoritmo=='KNeighborsRegressor':
        reg=KNeighborsRegressor(n_neighbors=vecinos)
    elif algoritmo=='RandomForestRegressor':
        reg=RandomForestRegressor(n_estimators=estimadores)
    return make_reduction(reg,strategy=estrategia, window_length=ventana)

# 5. Ajuste del Modelo

In [ ]:
def ajuste(predictor, y_train):
    return predictor.fit(y_train)

# 6. Predicción del Modelo

In [ ]:
def prediccion(predictor, horiz_prediccion):
    return predictor.predict(horiz_prediccion)

# 7. Evaluación del Modelo

In [ ]:
def evaluacion_modelo(test,pred):
    mape = mean_absolute_percentage_error(test, pred, symmetric=False)
    # smape = mean_absolute_percentage_error(test, pred) #symmetric default=True
    mae = mean_absolute_error(test, pred) 
    rmse = mean_squared_error(test, pred, square_root=True) # square_root=True RMSE , false MSE)
    mse = mean_squared_error(test, pred, square_root=False) # square_root=True RMSE , false MSE)
    return [mape, mae, rmse, mse]

# MODELOS ESTADÍSTICOS DE FORECASTING (SERIES TEMPORALES)

## Definir horizonte de predicción e índice

In [ ]:
tam_ypred = 12
indice='spei'

In [ ]:
'''cargar archivos con datos de cluster'''
cluster = pd.read_csv('https://raw.githubusercontent.com/MandbeZ/TFM_sequia/main/datos/spi_spei/cluster_4.csv',  sep = ',', usecols = {'id', 'cluster', 'estacion'})

'''-- Declara DF para almacenar los y_pred,
      DF para almacenar los datos de la evaluacion'''
df_predts = pd.DataFrame()
df_evalts = pd.DataFrame(index = ['MAPE','MAE','RMSE','MSE'])
columnas = []


'''Escala representa la escala temporal de SPI y SPEI = 3,6,12'''
for escala in [3,6,12]:
        
    '''1. Cargar archivo SPI-SPEI'''
    cad = indice+str(escala)
    datos = cargar_datos('indices_' +cad+ '.csv')

    '''2. Procesamiento de datos'''
    datos_procesados = procesa_datos(datos)

    '''-- Normalizar datos'''
    datos_normalizados = normalizar_datos(datos_procesados)
    
    '''definir los clusters'''
    lista_cluster = sorted(cluster['cluster'].unique())
    
    '''i representa el número de cluster'''
    for i in  lista_cluster:
        print(f'escala {escala} - cluster {i}')
        '''-- Trabajar con un cluster'''
        serie = gen_cluster(cluster, i, datos_normalizados)

        '''3. Datos de entrenamiento y prueba'''
        y_train, y_test = dividir_datos(serie, tamanio = tam_ypred)

        '''Definir el horizonte de predicción'''
        horizonte_prediccion = horizon_prediccion(y_test)

        '''4. Configurar algoritmo, valores ['naive', 'arima', 'ets']'''
        predictor = config_algoritmo('arima', estacionalidad = 12, ventana = 48, estrategia = 'mean')

        '''5. Ajuste modelo'''
        predictor = ajuste(predictor, y_train)

        '''6. Predicción'''
        y_pred = prediccion(predictor, horizonte_prediccion)

        '''7. Evaluación'''
        datos_eval = evaluacion_modelo(y_test, y_pred)

        '''-- Almacenar todos los y_pred, datos_eval en DFs y nombre de columnas'''
        df_predts = pd.concat([df_predts, y_pred.reset_index(drop=True)], axis = 1)
        df_evalts[indice+str(escala)+'_c'+str(i)] = datos_eval
        columnas.append(indice+str(escala)+'_c'+str(i))

'''Cambiar el nombre de las columnas'''
df_predts.columns = columnas



escala 3 - cluster 0
escala 3 - cluster 1
escala 3 - cluster 2
escala 3 - cluster 3


ValueError: ignored

In [ ]:
graficar_pred_est(df_predts, 'Modelo Arima')

In [ ]:
df_predts

In [ ]:
df_evalts

# ALGORITMOS DE REGRESIÓN DE APRENDIZAJE AUTOMÁTICO

In [ ]:
cluster = pd.read_csv('https://raw.githubusercontent.com/MandbeZ/TFM_sequia/main/datos/spi_spei/cluster_4.csv',  sep = ',', usecols = {'id', 'cluster', 'estacion'})

'''-- Declara DF para almacenar los y_pred,
      DF para almacenar los datos de la evaluacion'''
df_pred = pd.DataFrame()
df_eval = pd.DataFrame(index = ['MAPE','MAE','RMSE','MSE'])
columnas = []

'''Definir el horizonte de la prediccion'''
tam_ypred = 12

for escala in [3,6,12]:
        
    '''1. Cargar archivo'''
    cad = 'spi'+str(escala)
    datos = cargar_datos('indices_' +cad+ '.csv')

    '''2. Procesamiento de datos'''
    datos_procesados = procesa_datos(datos)

    '''-- Normalizar datos'''
    datos_normalizados = normalizar_datos(datos_procesados)    
    
    '''definir los clusters'''
    lista_cluster = sorted(cluster['cluster'].unique())
    for i in lista_cluster:

        '''-- Trabajar con un cluster'''
        serie = gen_cluster(cluster, i, datos_normalizados)

        '''3. Datos de entrenamiento y prueba'''
        y_train, y_test = dividir_datos(serie, tamanio = tam_ypred)

        '''Definir el horizonte de predicción'''
        horizonte_prediccion = horizon_prediccion(y_test)

        '''4. Configurar algoritmo de regresión, valores ['DecisionTreeRegressor','RandomForestRegressor', 'KNeighborsRegressor']'''
        predictor = config_algoritmoReg('RandomForestRegressor',  ventana = 48, estrategia = 'recursive',estimadores=30, vecinos=11)

        '''5. Ajuste modelo'''
        predictor = ajuste(predictor, y_train)

        '''6. Predicción'''
        y_pred = prediccion(predictor, horizonte_prediccion)

        '''7. Evaluación'''
        datos_eval = evaluacion_modelo(y_test, y_pred)

        '''-- Almacenar todos los y_pred, datos_eval en DFs y nombre de columnas'''
        df_pred = pd.concat([df_pred, y_pred.reset_index(drop=True)], axis = 1)
        df_eval[indice+str(escala)+'_c'+str(i)] = datos_eval
        columnas.append(indice+str(escala)+'_c'+str(i))
        
'''Cambiar el nombre de las columnas'''
df_pred.columns = columnas



In [ ]:
graficar_pred_est(df_pred, 'Modelo RandomForestRegressor')

In [ ]:
df_pred

In [ ]:
df_eval

In [ ]:
t1 = pd.Timestamp.now()
t1-t0